In [2]:
!pip install absl-py langdetect nltk immutabledict datasets bitsandbytes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from tqdm import tqdm
import json

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the dataset
dataset = load_dataset("google/IFEval")

# Model name
model_name = "Qwen/Qwen2.5-7B"

# Load the tokenizer for Mistral
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_eos_token=True,
    use_fast=True,
    padding_side='left'
)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token

# Quantization configuration using bitsandbytes library
# Config for 8 bit quantization
nf8_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_use_double_quant=True,
    bnb_8bit_compute_dtype=torch.bfloat16
)
# Step 2: Get trained LORA and BNB model
lora_location = "../final_model/"
model = AutoModelForCausalLM.from_pretrained(
    lora_location, 
    torch_dtype=torch.float16,  # Use float16 for mixed precision training
    device_map=device,  # Distribute the model automatically across GPUs
    quantization_config=nf8_config,  # Use the bitsandbytes quantization config
)
model.config.pad_token_id = tokenizer.pad_token_id  # Set the model's padding token ID

# Disable gradients to save memory and computation
model.eval()
torch.set_grad_enabled(False)  # Disable gradient computation globally

# Prepare the output file
output_dir = "../data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "input_response_data.jsonl")

# Batch processing
batch_size = 8  # Adjust based on your GPU memory capacity
max_length = 512  # Limit output length to avoid excessive memory usage
"""
with open(output_file, 'w') as f:
    # Process in batches
    for i in tqdm(range(0, len(dataset['train']), batch_size), desc="Processing Batches", unit="batch"):
        try:
            if (i + batch_size) > len(dataset['train']):
                batch_prompts = dataset['train']['prompt'][i:len(dataset['train'])]
            else:
                batch_prompts = dataset['train']['prompt'][i:i + batch_size]
            # Tokenize inputs
            inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(device)
            
            # Generate responses
            with torch.no_grad():  # Ensure gradients are disabled during generation
                if max_length:
                    outputs = model.generate(**inputs, max_new_tokens=max_length)
                else:
                    outputs = model.generate(**inputs)
            # Decode responses
            responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

            # Write each response directly to the file
            for prompt, response in zip(batch_prompts, responses):
                f.write(json.dumps({"prompt": prompt, "response": response}) + '\n')
        except Exception as e:
            print(f"Error processing batch {i}: {e}")"""

print(f"Responses saved to {output_file}")


Unused kwargs: ['bnb_8bit_quant_type', 'bnb_8bit_use_double_quant', 'bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Responses saved to data/input_response_data.jsonl
